<style>
@import url(https://www.numfys.net/static/css/nbstyle.css);
</style>
<a href="https://www.numfys.net"><img class="logo" /></a>

# Дискретное преобразование Фурье и быстрое преобразование Фурье

### Examples - Astrophysics
<section class="post-meta">
By Jonas Tjemsland, Andreas Krogen, Håkon Ånes and Jon Andreas Støvneng
</section>
Last edited: March 11th 2018 
___

В этом блокноте дается краткое введение в дискретное преобразование Фурье (DFT, ДПФ) и быстрое преобразование Фурье (FFT, БПФ). Реализован алгоритм БПФ radix-2 Cooley-Tukey, и ближе к концу объясняется физический смысл.

Эти концепции имеют широкую область применения во многих различных областях физики и математики, таких как обработка сигналов, фильтрация звука и изображений, сжатие данных, уравнения в частных производных и умножение больших целых чисел.

Прежде чем читать данный материал, нужно иметь представление о том, что такое [преобразование Фурье и ряд Фурье](https://habr.com/ru/post/196374/). 

Мы начинаем с импорта необходимых пакетов.

In [ ]:
import numpy as np
from numpy import random as rnd
import timeit
from scipy import fftpack as fft

## Дискретное Преобразование Фурье (DFT)

Пусть $\vec x = [x_0,x_1,...,x_{n-1}]$ - вектор $n$ комплексных (или вещественных) элементов. DFT $\vec x$ - это комплексный вектор $\vec y = [y_0,y_1,...,y_{n-1}]$, где элементы определяются как
$$y_k=\sum_{j=0}^{n-1}x_j\omega^{k\cdot j},$$
где $\omega = \exp(-2\pi i /n)$ ($i$ - мнимая единица) [1]. 

In [ ]:
def DFT(x):
    """ Вычисляет одномерное дискретное преобразование Фурье вектора.
    
    :x: double arr. Вектор, который преобразуется.
    :returns: double arr. Преобразование Фурье x.
    """
    n = len(x)
    y = [0]*n
    omega = np.exp(-2.0j*np.pi/n)
    for k in range(0,n):
        y[k] = np.sum(x*omega**(np.arange(0,n)*k))
    return y

Очевидно, что обратное DFT задается
$$x_k = \sum_{j=0}^{n-1} y_j\omega^{k\cdot j},$$
где $\omega = \exp(2\pi i/n)$.

In [ ]:
def inverseDFT(y):
    """ Вычисляет обратное одномерное дискретное Фурье преобразование вектора.
    
    :y: double arr. Вектор, который преобразуется.
    :returns: double arr. Обратное преобразование Фурье y.
    """
    n = len(y)
    x = [0]*n
    omega = np.exp(2.0j*np.pi/n)
    for k in range(0,n):
        x[k] = np.sum(y*omega**(np.arange(0,n)*k))/float(n)
    return x

Давайте попробуем на небольшом примере, где мы просто преобразуем туда и обратно произвольный вектор.

In [ ]:
# Определение массива, который преобразуется.
x = rnd.randint(8,size=8)
print('x =', x)
# Преобразование Фурье
y = DFT(x)
print('y =', np.round(y,2))
# Обратное преобразование Фурье. 
x = inverseDFT(y)
print('x =', np.round(x,2))

Как вы уже могли заметить, этот DFT-алгоритм довольно неэффективен. Существует множество шагов, которые выполняются более одного раза, и, как следствие, сложность этого алгоритма составляет $\mathcal O(n^2)$. 

## Быстрое преобразование Фурье (FFT)
Алгоритмы БПФ используют симметрии и схожесть некоторых операций. Далее мы обсудим алгоритм Кули–Тьюки [2].

Предположим, что $N$ является составным. Это означает, что $N=n_1\cdot n_2$, где $N$, $n_1$ и $n_2$ - целые числа. Перепишем эти два показателя следующим образом 
$$k=n_2k_1+k_2,$$
$$j = n_1j_2 + j_1,$$
где $k_{1,2} = 0,1,...,n_{1,2}-1$ и $j_{1,2} = 0,1,...,j_{1,2}-1$. Если мы подставим эти новые показатели в DFT, некоторые перекрестные слагаемые исчезнут, и конечный результат будет
$$y_{n_2k_1+k_2}=\sum_{j_1=0}^{n_1-1}\sum_{j_2=0}^{n_2-1}x_{n_1j_2+n_1}\exp\left[\frac{-2\pi i}{n_1n_2}(n_1j_2+j_1)(n_2k_1+k_2)\right]$$
$$=\sum_{j_1=0}^{n_1-1}\exp\left[-\frac{2\pi i}{n}j_1k_2\right]\left(\sum_{j_2=0}^{n_2-1}x_{n_1j_2+j_1}\exp\left[-\frac{2\pi i}{n_2}j_2k_2\right]\right)\exp\left[-\frac{2\pi i}{n_1}j_1k_1\right].$$
В этом уравнении каждая внутренняя сумма является DFT размером $n_2$, а каждая внешняя сумма-DFT размером $n_1$. Это дает рекурсивную формулу для вычисления DFT, которая более подробно описана в [1] и [4]. Для простоты воспользуемся алгоритмом *radix-2*. Сложность алгоритма FFT составляет $\mathcal O (n\log n)$, что делает его почти линейным для больших наборов данных!

In [ ]:
def CooleyTukeyRadix2FFT(x):
    """ Вычисляет одномерное дискретное преобразование Фурье вектора с использованием 
    алгоритма БПФ Кули-Тьюки radix-2. 
    Преобразуемый вектор должен иметь степень 2 числа элементов.
    
    :x: double arr. Вектор, который преобразуется.
    :returns: double arr. Преобразование Фурье x.
    """
    # Проверим, является ли n степенью 2.
    if ( len(x) & (len(x) - 1)):
        raise Exception("Число элементов в x должно быть степенью 2!")
    # Рекурсивная формула для вычисления БПФ.
    def foo(x):
        n = len(x)
        if n == 1:
            y = x
        else:
            y2 = foo(x[0:n:2])
            y1 = foo(x[1:n + 1:2])
            d = np.exp(-2j*np.pi/n)**np.arange(0,n/2)
            y = np.append(y2 + d*y1,y2 - d*y1)
        return y
    return foo(x)

def inverseCooleyTukeyRadix2FFT(y):
    """ Вычисляет одномерное обратное дискретное преобразование Фурье
    вектора с использованием алгоритма БПФ Кули-Тьюки radix-2. Размер вектора
    должен быть степенью 2.
    Parameters:
        x: double arr. Вектор, который преобразуется.
    Returns:
        y: double arr. Преобразование Фурье x.
    """
    # CПроверим, является ли n степенью 2.
    if (len(y) & (len(y) - 1)):
        raise Exception("Число элементов в x должно быть степенью 2!")
    # Rэкурсивная формула для расчета БПФ.
    def foo(y):
        n = len(y)
        if n == 1:
            x = y
        else:
            x2 = foo(y[0:n:2])
            x1 = foo(y[1:n + 1:2])
            d = np.exp(2j*np.pi/n)**np.arange(0,n/2)
            x = np.append(x2 + d*x1,x2 - d*x1)
        return x
    return foo(y)/len(y)

Давайте попробуем на небольшом примере, где мы просто преобразуем туда и обратно произвольный вектор, как и раньше.

In [ ]:
# Определение преобразуемого массива.
x = rnd.randint(10,size=8)
print('x =', x)
# Преобразование Фурье.
y = CooleyTukeyRadix2FFT(x)
print('y =', np.round(y,2))
# Обратное преобразование Фурье. 
x = inverseCooleyTukeyRadix2FFT(y)
print('x =', np.round(x,2))

Чтобы продемонстрировать превосходство БПФ, мы вычисляем преобразование Фурье для гораздо большего набора данных. Давайте также сравним с функцией fft из scipy.fftpack.

In [ ]:
x = rnd.rand(2048)
# Время цикла для DFT, CooleyTukeyRadix2FFT и scipy.fftpack.fft.
%timeit y = DFT(x)
%timeit y = CooleyTukeyRadix2FFT(x)
%timeit y = fft.fft(x)

## Физический смысл

Прямое дискретное преобразование Фурье ставит в соответствие временной функции, которая определена N-точками измерений на заданном временном интервале, другую функцию, которая определена на частотном интервале. Таким образом, физический смысл дискретного преобразования Фурье состоит в том, чтобы представить некоторый дискретный сигнал в виде суммы гармоник. Параметры каждой гармоники вычисляются прямым преобразованием, а сумма гармоник - обратным.

Чтобы проиллюстрировать это, нам нужно выяснить, что физически означает формула DFT. Мы начнем с того, что перепишем ее как
$$x_k=\sum_{j=0}^{n-1}y_j\exp\left(2\pi i\frac{k}{n\Delta t}j\Delta t\right).$$
Выражение говорит нам просто о том, что $\vec x$ - это суперпозиция экспоненциальных функций с различными частотами $f_j = \frac{j}{n\Delta t}$ и амплитудами $y_j$. Поэтому мы можем рассматривать величину амплитуды $|y_k|^2$ как меру "веса частоты $f_j$" в $\vec x$!

## Многомерное DFT
Положим $\vec j = (j_1,j_2,...,j_d)$ и $\vec k = (k_1,k_2,...,k_d)$ являются $d$-мерными векторами с индексацией от $\vec 0$ до $\vec n-1 = (n_1-1,n_2,...,n_d-1)$. Тогда $d$-мерное DFT задается
$$y_\vec{k}=\sum_{\vec j=\vec 0}^{\vec n-1}x_\vec{j}\exp\left[-2\pi\vec k\cdot\vec \xi\right],$$
где $\vec \xi$ является поэлементным делением $(j_1/n_1,...,j_d/n_d)$ [4]. Например, двумерный DFT задается
$$\vec y_{k_1,k_2}=\sum_{j_1=0}^{n_1-1}\sum_{j_2=0}^{n_2-1}x_{j_1,j_2}\exp\left[-2\pi i\left(\frac{ k_1j_1}{n_1}+\frac{k_2j_2}{n_2}\right)\right].$$

Ссылки:  
[1] T. Sauer: Numerical Analysis, second edition, Pearson 2014  
[2] James W. Cooley and John W. Tukey: An Algorithm for the Machine Calculation of Complex Fourier Series, Math. Comp. 19 (1965), p. 297-301  
[3] Wikipedia: https://en.wikipedia.org/wiki/Cooley%E2%80%93Tukey_FFT_algorithm, 03.28.2016 (acquired: April 2016)  
[4] Wikipedia: https://en.wikipedia.org/wiki/Discrete_Fourier_transform, 04.28.2016 (acquired: April 2016)

Check out the FFT pack in SciPy:
http://docs.scipy.org/doc/scipy/reference/fftpack.html  